In [1]:
from wrds2pg import get_process

In [2]:
tz = "America/New_York"
schema = "audit"
table_name = "feed20_nt"
obs = 20

tz_vars = ["file_accepted", "ac_file_accepted"]
keep = "nt_notify_key " + ' '.join(tz_vars)
def tz_str(i, var):
    temp =  f"        temp_var_{i} = input({var}, YMDDTTM.);\n"
    temp += f"        drop {var};\n"
    temp += f"        rename temp_var_{i} = {var};\n"
    return temp

if tz_vars:
    tz_str_list = [tz_str(i, var) for i, var in enumerate(tz_vars)]
    tz_str = '\n'.join(tz_str_list)
    timestamps_list = [ '        attrib ' + var + ' format=E8601DX.;'
                                       for var in tz_vars]
    timestamps_str = '\n'.join(timestamps_list)

print(tz_str)

        temp_var_0 = input(file_accepted, YMDDTTM.);
        drop file_accepted;
        rename temp_var_0 = file_accepted;

        temp_var_1 = input(ac_file_accepted, YMDDTTM.);
        drop ac_file_accepted;
        rename temp_var_1 = ac_file_accepted;



In [3]:
if obs:
    obs_str = f"obs={obs}"

if keep:
    keep_str = f"keep={keep}"

sas_code = f"""
    options nosource nonotes;
    options timezone='{tz}';
    
    data temp;
        set {schema}.{table_name}({obs_str} {keep_str});
{tz_str}
    run;

    proc datasets lib=work;
        modify temp;
{timestamps_str}
    run;

    proc export data=temp
        outfile=stdout 
        dbms=csv;
    run;
    
    /*
    proc contents data=temp;
    run;
    */
"""

In [4]:
print(sas_code)


    options nosource nonotes;
    options timezone='America/New_York';
    
    data temp;
        set audit.feed20_nt(obs=20 keep=nt_notify_key file_accepted ac_file_accepted);
        temp_var_0 = input(file_accepted, YMDDTTM.);
        drop file_accepted;
        rename temp_var_0 = file_accepted;

        temp_var_1 = input(ac_file_accepted, YMDDTTM.);
        drop ac_file_accepted;
        rename temp_var_1 = ac_file_accepted;

    run;

    proc datasets lib=work;
        modify temp;
        attrib file_accepted format=E8601DX.;
        attrib ac_file_accepted format=E8601DX.;
    run;

    proc export data=temp
        outfile=stdout 
        dbms=csv;
    run;
    
    /*
    proc contents data=temp;
    run;
    */



In [5]:
p = get_process(sas_code, wrds_id="iangow")

In [6]:
p.readlines()

['NT_NOTIFY_KEY,file_accepted,ac_file_accepted\n',
 '18350,2004-05-17T13:19:23-04:00,2004-05-24T13:33:35-04:00\n',
 '15476,2004-08-16T15:01:27-04:00,2004-08-18T11:56:04-04:00\n',
 '4370,2005-04-01T09:39:51-05:00,2005-05-19T05:55:59-04:00\n',
 '4246,2005-05-17T10:46:11-04:00,2005-06-23T10:47:44-04:00\n',
 '2271,2005-08-16T10:41:29-04:00,2005-09-02T12:52:40-04:00\n',
 '754,2005-11-15T11:35:43-05:00,2005-12-02T09:55:40-05:00\n',
 '13211,2006-05-16T06:56:34-04:00,2006-05-21T20:00:00-04:00\n',
 '26155,2006-08-14T15:27:39-04:00,2006-08-18T12:18:15-04:00\n',
 '33292,2006-11-15T06:40:33-05:00,2006-11-20T07:37:08-05:00\n',
 '48220,2007-04-03T08:32:27-04:00,2007-04-17T09:45:19-04:00\n',
 '53008,2007-05-15T20:00:00-04:00,2007-05-21T07:16:38-04:00\n',
 '54900,2007-08-14T20:00:00-04:00,2007-08-20T08:40:14-04:00\n',
 '56535,2007-11-15T08:57:11-05:00,2007-12-07T12:48:13-05:00\n',
 '58210,2008-03-31T09:40:51-04:00,2008-05-15T10:55:05-04:00\n',
 '59638,2008-05-15T09:15:04-04:00,2008-05-19T17:59:09-04:0